# Composition

This section contains higher-level components that combine other arbitrary systems (e.g. external APIs and services) and/or LangChain primitives together.

A good primer for this section would be reading the sections on LangChain Expression Language and becoming familiar with constructing sequences via piping and the various primitives offered.

The components covered in this section are:

# 1. Tools

Tools provide an interface for LLMs and other components to interact with other systems. Examples include Wikipedia, a calculator, and a Python REPL

Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:

The name of the tool
A description of what the tool is
JSON schema of what the inputs to the tool are
The function to call
Whether the result of a tool should be returned directly to the user
It is useful to have all this information because this information can be used to build action-taking systems! The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take, and then the function to call is equivalent to taking that action.

The simpler the input to a tool is, the easier it is for an LLM to be able to use it. Many agents will only work with tools that have a single string input. 

In [4]:
!pip install wikipedia

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11704 sha256=8f374f006a85cf05d7ac68027c639dd9dde2e69e0080dd9e20763195db143d56
  Stored in directory: c:\users\janu\appdata\local\pip\cache\wheels\8f\ab\cb\45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
tool.name

'wikipedia'

In [4]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [5]:
tool.args

{'query': {'title': 'Query', 'type': 'string'}}

In [6]:
tool.run({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

### Yahoo Finance tool

In [13]:
!pip install --upgrade --quiet  yfinance

In [8]:
import os

#Setting Environment variable
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

In [9]:
from langchain_core.messages import AIMessage,SystemMessage,HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings ##embedding type

### Defining a chat model

In [10]:
chat_model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

### Defining a embeddings

In [12]:
embeddings = AzureOpenAIEmbeddings(model = os.environ["LLM_EMBEDDING_MODEL"],)

### Defining Tools

In [14]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool

tools = [YahooFinanceNewsTool()]
agent_chain = initialize_agent(tools,chat_model,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True,)

C:\Users\JANU\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [16]:
agent_chain.invoke(
    "What happens today with Microsoft stocks?",
)



> Entering new AgentExecutor chain...
I should check the latest news about Microsoft stocks.
Action: yahoo_finance_news
Action Input: MSFT
Observation: Nasdaq 100 Breaks 3-Day Losing Streak As Tech Stocks Rebound Ahead Of Earnings Week
The Nasdaq 100 index, as tracked by the Invesco QQQ Trust (NASDAQ:QQQ), ended Monday 1% higher, breaking a three-day losing streak as traders prepare for a crucial week of earnings reports from some of the tech sector’s most influential names. This week’s earnings lineup features four Magnificent Seven giants like Tesla Inc. (NASDAQ:TSLA), which will report on Tuesday, followed by Meta Platforms Inc. (NASDAQ:META) on Wednesday, and both Microsoft Corp. (NYSE:MSFT) and Alphabet Inc. (NASDAQ:GOOGL
Thought:Microsoft stocks are performing well today as the Nasdaq 100 index ended 1% higher.
Final Answer: Microsoft stocks ended higher today.

> Finished chain.


{'input': 'What happens today with Microsoft stocks?',
 'output': 'Microsoft stocks ended higher today.'}


To see more - https://python.langchain.com/docs/integrations/tools/

We can also create a custom tool based on customer requirement.

# 2. Agents 

Agents use a language model to decide actions to take, often defined by a tool. They require an executor, which is the runtime for the agent. The executor is what actually calls the agent, executes the tools it chooses, passes the action outputs back to the agent, and repeats. The agent is responsible for parsing output from the previous results and choosing the next steps.

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

let’s build an agent that has two tools: one to look things up online, and one to look up specific data that we’ve loaded into a index.

In [22]:
#!pip install -U langchain-community tavily-python

In [17]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [19]:
search = TavilySearchResults()

In [20]:
response = search.invoke("what is today's weather in Chennai with the date")

In [21]:
print(response)

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Chennai', 'region': 'Tamil Nadu', 'country': 'India', 'lat': 13.08, 'lon': 80.28, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1713862679, 'localtime': '2024-04-23 14:27'}, 'current': {'last_updated_epoch': 1713861900, 'last_updated': '2024-04-23 14:15', 'temp_c': 37.0, 'temp_f': 98.6, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 130, 'wind_dir': 'SE', 'pressure_mb': 1006.0, 'pressure_in': 29.71, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 42, 'cloud': 25, 'feelslike_c': 48.2, 'feelslike_f': 118.8, 'vis_km': 7.0, 'vis_miles': 4.0, 'uv': 8.0, 'gust_mph': 19.0, 'gust_kph': 30.6}}"}, {'url': 'https://www.timeanddate.com/weather/india/chennai/hourly', 'content': 'Hour-by-Hour Forecast for Chennai, Tamil Nadu, India. Time/General. Weather. Time Zone. DST Changes. Sun & Moon. Weather To

In [12]:
#!pip install faiss-cpu

In [22]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, embeddings,
)
retriever = vector.as_retriever()

In [23]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='# The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from \'langsmith\';import { runOnDataset } from \'langchain/smith\';import { EvaluationResult } from \'langsmith/evaluation\';const client = new Client();// Define dataset: these are your test casesconst datasetName = "Sample Dataset";const dataset = await client.createDataset(datasetName, {    description: "A sample dataset in LangSmith."});await client.createExamples({    inputs: [        { postfix: "to LangSmith" },        { postfix: "to Evaluations in LangSmith" },    ],    outputs: [        { output: "Welcome to LangSmith" },        { output: "Welcome to Evaluations in LangSmith" },    ],    datasetId: dataset.id,});// Define your evaluatorconst exactMatch = async ({ run, example }: {

In [24]:
from langchain.tools.retriever import create_retriever_tool

In [25]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [26]:
tools = [search, retriever_tool]

## Create the agent

Now that we have defined the tools, we can create the agent. We will be using an OpenAI Functions agent - for more information on this type of agent, as well as other options, see this guide.

First, we choose the LLM we want to be guiding the agent.

In [27]:
from langchain_openai import AzureChatOpenAI

chat_model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [20]:
!pip install langchainhub

Defaulting to user installation because normal site-packages is not writeable


In [28]:
from langchain import hub

# Get the prompt to use - you can modify this! - https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [29]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(chat_model, tools, prompt)

In [30]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Run an agent

In [31]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [32]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith testing'}`


Getting started with LangSmith | ğŸ¦œï¸�ğŸ› ï¸� LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroductionâ€‹LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install LangSmithâ€‹We offer Python and Typescript SDKs for all your LangSmith needs.PythonTypeScriptpip install -U langsmithyarn add langchain langsmithCreate an API keyâ€‹To create an API key head to the setting pages. Then click Create API Key.Setup your environmentâ€‹Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># T

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing by providing a platform for building production-grade language model applications. It allows you to closely monitor and evaluate your application, ensuring that it performs as expected. LangSmith offers tracing capabilities to track the behavior of your application, evaluation capabilities to assess its performance, and prompt management tools for efficient testing. Additionally, LangSmith supports workflows at each stage of the language model application lifecycle, making it easier to test and iterate on your application.'}

In [33]:
agent_executor.invoke({"input": "whats the weather in chennai?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Chennai'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Chennai', 'region': 'Tamil Nadu', 'country': 'India', 'lat': 13.08, 'lon': 80.28, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1713862999, 'localtime': '2024-04-23 14:33'}, 'current': {'last_updated_epoch': 1713862800, 'last_updated': '2024-04-23 14:30', 'temp_c': 37.0, 'temp_f': 98.6, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 130, 'wind_dir': 'SE', 'pressure_mb': 1006.0, 'pressure_in': 29.71, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 42, 'cloud': 25, 'feelslike_c': 48.2, 'feelslike_f': 118.8, 'vis_km': 7.0, 'vis_miles': 4.0, 'uv': 8.0, 'gust_mph': 19.0, 'gust_kph': 30.6}}"}, {'url': 'https://www.mykhel.com/cricket/csk-vs-lsg-ipl-2024-ma-chidambaram-stadium

{'input': 'whats the weather in chennai?',
 'output': 'The current weather in Chennai is partly cloudy with a temperature of 37°C (98.6°F). The wind speed is 19.1 km/h coming from the southeast. The humidity is 42% and the visibility is 7.0 km.'}

### Adding in memory

In [34]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [35]:
from langchain_core.messages import AIMessage, HumanMessage

In [36]:
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Bob.'}

In [37]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [38]:
message_history = ChatMessageHistory()

In [39]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [40]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [41]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

There are several key components here: a string.

## Schema
LangChain has several abstractions to make working with agents easy.

### 1. AgentAction

This is a dataclass that represents the action an agent should take. It has a tool property (which is the name of the tool that should be invoked) and a tool_input property (the input to that tool)

### 2. AgentFinish

This represents the final result from an agent, when it is ready to return to the user. It contains a return_values key-value mapping, which contains the final agent output. Usually, this contains an output key containing a string that is the agent's response.

### 3. Intermediate Steps

These represent previous agent actions and corresponding outputs from this CURRENT agent run. These are important to pass to future iteration so the agent knows what work it has already done. This is typed as a List[Tuple[AgentAction, Any]]. Note that observation is currently left as type Any to be maximally flexible. In practice, this is often a string.

## Agent Types
This categorizes all the available agents along a few dimensions.

### Intended Model Type

Whether this agent is intended for Chat Models (takes in messages, outputs message) or LLMs (takes in string, outputs string). The main thing this affects is the prompting strategy used. You can use an agent with a different type of model than it is intended for, but it likely won't produce results of the same quality.

### Supports Chat History

Whether or not these agent types support chat history. If it does, that means it can be used as a chatbot. If it does not, then that means it's more suited for single tasks. Supporting chat history generally requires better models, so earlier agent types aimed at worse models may not support it.

### Supports Multi-Input Tools

Whether or not these agent types support tools with multiple inputs. If a tool only requires a single input, it is generally easier for an LLM to know how to invoke it. Therefore, several earlier agent types aimed at worse models may not support them.

### Supports Parallel Function Calling

Having an LLM call multiple tools at the same time can greatly speed up agents whether there are tasks that are assisted by doing so. However, it is much more challenging for LLMs to do this, so some agent types do not support this.

### Required Model Params

Whether this agent requires the model to support any additional parameters. Some agent types take advantage of things like OpenAI function calling, which require other model parameters. If none are required, then that means that everything is done via prompting

## Creating a Custom agent

In [45]:
#Defining the LLM/Chat Model

chat_model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [46]:
#Defining a tool

from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

In [48]:
#assiging the function to an object
tools = [get_word_length]

In [49]:
#Creating a prompt

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [50]:
#binding the tool to llm
llm_with_tools = chat_model.bind_tools(tools)

In [54]:
#create a chat agent

from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [55]:
#creating agent executor

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [56]:
#invoking the agent
list(agent_executor.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5There are 5 letters in the word "eudca".

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_0WnoGyGfwZroxOT6Q2N741NI', 'function': {'arguments': '{\n  "word": "eudca"\n}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-767da9aa-6ab0-4b30-b746-c3389cc4c87b', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'call_0WnoGyGfwZroxOT6Q2N741NI'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{\n  "word": "eudca"\n}', 'id': 'call_0WnoGyGfwZroxOT6Q2N741NI', 'index': 0}])], tool_call_id='call_0WnoGyGfwZroxOT6Q2N741NI')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_0WnoGyGfwZroxOT6Q2N741NI', 'function': {'arguments': '{\n  "word": "eudca"\n}', 'name': 'get_word_length'}, 'type': 'function'}

In [57]:
chat_model.invoke("How many letters in the word educa")

AIMessage(content='The word "educa" has 5 letters.', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-6431ed95-e642-4668-a96e-657b11928d86-0')

# 3. Chains

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.

LCEL is great for constructing your own chains, but it’s also nice to have chains that you can use off-the-shelf. There are two types of off-the-shelf chains that LangChain supports:

Chains that are built with LCEL. In this case, LangChain offers a higher-level constructor method. However, all that is being done under the hood is constructing a chain with LCEL.

[Legacy] Chains constructed by subclassing from a legacy Chain class. These chains do not use LCEL under the hood but are rather standalone classes.

We are working creating methods that create LCEL versions of all chains. We are doing this for a few reasons.

Chains constructed in this way are nice because if you want to modify the internals of a chain you can simply modify the LCEL.

These chains natively support streaming, async, and batch out of the box.

These chains automatically get observability at each step.

This page contains two lists. First, a list of all LCEL chain constructors. Second, a list of all legacy Chains.

In [66]:
!pip install -U langchain langchain-community

Defaulting to user installation because normal site-packages is not writeable


In [69]:
from langchain_openai import AzureChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [70]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "What are everyone's favorite colors:\n\n{context}")]
)
llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)
chain = create_stuff_documents_chain(llm, prompt)

docs = [
    Document(page_content="Jesse loves red but not yellow"),
    Document(page_content = "Jamal loves green but not as much as he loves orange")
]

In [71]:
chain.invoke({"context": docs})

"Jesse's favorite color is red.\nJamal's favorite color is orange."